<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from collections import Counter
import random
import time
import math

def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    duracion = time.perf_counter() - inicio
    return resultado, duracion

def obtener_fpm_dict(df_fpm):
    df_fpm_fpm = df_fpm[df_fpm['Tema'] == 'FPM']
    return dict(zip(df_fpm_fpm['Datos'], df_fpm_fpm['Valor']))

def calcular_TVD(dict_d1, dict_d2): # Distancia Total de Variación.
    soporte_total = set(dict_d1) | set(dict_d2)
    return 0.5 * sum(abs(dict_d1.get(k, 0) - dict_d2.get(k, 0)) for k in soporte_total)

# --------------------------- Pandas ---------------------------
def comparar_dfs(df_A, df_B):
    df_comparado = pd.merge(df_A[['Tema', 'Datos', 'Valor']],
                            df_B[['Datos', 'Valor']],
                            on='Datos',
                            how='outer',
                            suffixes=('_A', '_B'))

    df_comparado['Diferencia'] = (df_comparado['Valor_A'] - df_comparado['Valor_B']).abs()
    return df_comparado[['Tema', 'Datos', 'Valor_A', 'Valor_B', 'Diferencia']]

def crear_df_fpm(dict_fpm):
    soporte = list(dict_fpm.keys())
    data = [{
        'Tema': 'FPM',
        "Datos": x,
        "Valor": dict_fpm.get(x, 0)
    } for x in soporte]

    esp = sum(x * p for x, p in dict_fpm.items())
    var = sum((x - esp)**2 * p for x, p in dict_fpm.items())
    desv = var**0.5
    resumen = [
        {'Tema': 'Estadisticas', "Datos": "Esperanza", "Valor": esp},
        {'Tema': 'Estadisticas', "Datos": "Varianza", "Valor": var},
        {'Tema': 'Estadisticas', "Datos": "Desv. Est.", "Valor": desv},
    ]

    return pd.concat([pd.DataFrame(data), pd.DataFrame(resumen)], ignore_index=True)

def estimar_df_fpm(generador_de_muestras, n=100_000):
    muestras, tiempos = zip(*(medir_tiempo(generador_de_muestras) for _ in range(n)))

    frec = Counter(muestras)
    soporte = sorted(list(frec.keys()))

    dict_fpm_estimada = {x: frec.get(x, 0) / n for x in soporte}
    df_fpm_estimada = crear_df_fpm(dict_fpm_estimada)

    tiempo_promedio = sum(tiempos) / len(tiempos)
    df_tiempo_promedio = pd.DataFrame([{
        'Tema': 'Performance',
        "Datos": "Tiempo Promedio",
        "Valor": tiempo_promedio
    }])

    return pd.concat([df_fpm_estimada, df_tiempo_promedio], ignore_index=True)

# --------------------------- Utilidades ---------------------------

def ordenar_dict_fpm_por_probabilidad(dict_fpm):
  return dict(sorted(dict_fpm.items(), key=lambda x: x[1], reverse=True))

def calcular_dict_fda(dict_fpm):
  dict_fda = {}
  acc = 0
  for v, p in dict_fpm.items():
    acc += p
    dict_fda[v] = acc
  return dict_fda

# def crear_dict_fpm(soporte, fpm):
#   return {v: fpm(v) for v in soporte}

# dict_fpm = dict(gen_fpm)

def probabilidad(condicion, n):
    exitos = sum(condicion() for _ in range(n))
    return exitos / n

def get_gen_fpm_truncada(get_gen_fpm, condicion):
  list_fpm = list(get_gen_fpm())
  suma_probabilidades = sum(p for i, p in list_fpm if condicion(i))

  for i, p in list_fpm:
    if condicion(i):
      yield i, p / suma_probabilidades

# def TI(fpm): # soporte subarreglo de [0, 1, ..., inf]
#   u = random.random()
#   i = 0; i_fda = fpm(0)
#   while u >= i_fda:
#     i += 1; i_fda += fpm(i)
#   return i

def TI(dict_fda):
  u = random.random()
  for v, p in dict_fda.items():
    if u < p:
      return v

def AyR(dict_fpm_objetivo, fpm_propuesta, dict_fpm_propuesta, C):
    while True:
        muestra = fpm_propuesta()
        prob_objetivo = dict_fpm_objetivo.get(muestra, 0)
        prob_propuesta = dict_fpm_propuesta.get(muestra, 0)

        if prob_propuesta == 0:
            continue

        if random.random() < prob_objetivo / (C * prob_propuesta):
            return muestra

In [23]:
# Ejercicio 9. Implemente dos métodos para simular una variable geométrica Geom(p):
# a) Usando transformada inversa y aplicando la fórmula recursiva para P(X = i).
def get_gen_fpm_Geom(p): # de 1 a m
  i = 1; acc = 0
  while acc < 1 - 1e-10:
    fpm_i = p * (1 - p)**(i-1)
    yield i, fpm_i
    i += 1
    acc += fpm_i

P_GEOM = 0.2
dict_fpm_Geom = dict(get_gen_fpm_Geom(P_GEOM))
dict_fda_Geom = calcular_dict_fda(dict_fpm_Geom)
generador_de_muestras_TI_A = lambda: TI(dict_fda_Geom)

# Arriba seria a la vieja usansa, asi seria mejor:
generador_de_muestras_TI_B = lambda: int(math.log10(1 - random.random()) / math.log10(1 - P_GEOM)) + 1

df_A = estimar_df_fpm(generador_de_muestras_TI_A)
df_B = estimar_df_fpm(generador_de_muestras_TI_B)

# b) Simulando ensayos con probabilidad de éxito p hasta obtener un éxito.
def generador_de_muestras_con_ensayos_geom():
  fallos = 0
  while P_GEOM < random.random():
    fallos +=1
  return fallos + 1
df_C = estimar_df_fpm(generador_de_muestras_con_ensayos_geom)

print(calcular_TVD(dict_fpm_Geom, obtener_fpm_dict(df_B)))
comparar_dfs(crear_df_fpm(dict_fpm_Geom), df_B)

0.0055552527636569616


,Tema,Datos,Valor_A,Valor_B,Diferencia
0,FPM,1,2.000000e-01,1.990800e-01,0.000920
1,FPM,2,1.600000e-01,1.606400e-01,0.000640
2,FPM,3,1.280000e-01,1.285300e-01,0.000530
3,FPM,4,1.024000e-01,1.015000e-01,0.000900
4,FPM,5,8.192000e-02,8.027000e-02,0.001650
...,...,...,...,...,...
103,FPM,104,2.085925e-11,NaN,NaN
104,Estadisticas,Desv. Est.,4.472136e+00,4.496390e+00,0.024254
105,Estadisticas,Esperanza,5.000000e+00,5.018730e+00,0.018730
106,NaN,Tiempo Promedio,NaN,3.115298e-07,NaN
